In this tutorial, we show how to use the fSBI method proposed in the companion paper.  
We will be performing an additional fSBI round as follows:
- we start from the final posterior in the paper (the "plausible candidate rules", π<sub>3</sub>, either for MLP or polynomial plasticity rules).
- we will generate additional rules with more specific excitatory rates (between 5 and 10Hz, the initial filtering done in the paper includes rules with rates between 1 and 50Hz).
- we will train a new posterior p(θ|r<sub>exc</sub>)

In [ ]:
from fsbi.density_estimator import MakePosterior
from fsbi.utils import get_density_estim_data, resample_old_data, load_and_merge, apply_n_conditions, _make_unique_samples
from fsbi.prior import RestrictedPrior as Prior
from torch import FloatTensor as FT
import matplotlib.pyplot as plt
import torch
import numpy as np
import h5py
import argparse
import yaml
import pickle
from time import time

#### 1/ Choose samples from the polynomial search space (See Fig 2)

In [ ]:
round_name = "pi3_r5to10Hz"

# directory where all the rules simulated in the paper are stored, with all metrics computed.
data_path = "data_synapsesbi/bg_IF_EEEIIEII_6pPol/"
# name: bg: stability task (background inputs), IF: integrate and fire neuron model in Auryn,EE EI IE II recurrent synapses plastic, 6pPol: polynomial parmeterization with 6 params. 

# where to store the posterior object
runs_path = "runs_synapsesbi/bg_IF_EEEIIEII_6pPol/"

# simulation parameters for the auryn simulations, later down the road
path_to_sim_config = "tasks_configs/bg_IF_EEEIIEII_6pPol.yaml"
with open(path_to_sim_config, "r") as f:
    sim_params = yaml.load(f, Loader=yaml.Loader)
    
# the bounds for the plasticity parameters
# time constants are between 10ms and 100ms, other params between -2 and 2
lower_lim=torch.tensor([0.01, 0.01, -2., -2., -2., -2.,
            0.01, 0.01, -2., -2., -2., -2.,
            0.01, 0.01, -2., -2., -2., -2.,
            0.01, 0.01, -2., -2., -2., -2.])
upper_lim=torch.tensor([.1, .1, 2., 2., 2., 2.,
        .1, .1, 2., 2., 2., 2.,
        .1, .1, 2., 2., 2., 2.,
        .1, .1, 2., 2., 2., 2.])

# which metrics to train the posterior on
# fitting a posterior on too many metrics at the same time (especially similar/correlated metrics) generates worse quality posteriors, in our hands at least.
metrics = ["rate"]

In [ ]:
# here we are building the training set to fit the posterior on.
# there is a trade-off between 1/ including as many samples for training as possible and 
# 2/ allocating resources to fit the posterior to uninteresting regions of the parameter space (e.g. here we do not care about rules with high firing rates so need to include too many of them)

# first, we load all the rules simulated in the paper are stored, with all metrics computed. (pi0 -> pi3).
dataset_aux = load_and_merge(data_path, ("bg_IF_EEEIIEII_6pPol_all.npy",))

# then we define constrains to only keep rules relevant for our round of fSBI.

# all the conditions for plausibility from the paper
cond_cv = ("cv_isi", 0.7, 1000)
cond_sf = ("spatial_Fano", 0.5, 2.5)
cond_tf = ("temporal_Fano", 0.5, 2.5)
cond_ac = ("auto_cov", 0, 0.1)
cond_fft = ("fft", 0, 1)
cond_wb = ("w_blow", 0, 0.1)
cond_srt = ("std_rate_temporal", 0, 0.5)
cond_srs = ("std_rate_spatial", 0, 5)
cond_scv = ("std_cv", 0, 0.2)
cond_wc = ("w_creep", 0, 0.05)
cond_ri = ("rate_i", 1, 50)
cond_weef =("weef", 0 ,0.5)
cond_weif =("weif", 0 ,0.5)
cond_wief =("wief", 0 ,5)
cond_wiif =("wiif", 0 ,5)

# only keep the rules with exc rates between 3 and 15Hz for the training ([5,10]Hz + some leeway)
cond_r = ("rate", 3, 15)

condition = apply_n_conditions(dataset_aux, (cond_r,cond_ri,
            cond_wb,cond_wc,cond_weef,cond_weif, cond_wief, cond_wiif,
            cond_ac,cond_cv,cond_fft,cond_srt,cond_srs,cond_sf,cond_tf))

dataset = dataset_aux[condition]
print(str(np.sum(condition)) + "/" + str(len(condition)), "samples kept for training")

#### 1 bis/ Choose samples from the MLP search space (See Fig 4) 

This section is an alternative to the polynomial one above in case you want to use MLP rules.

In [ ]:
round_name = "pi3_r5to10Hz"
data_path = "data_synapsesbi/bg_CVAIF_EEIE_T4wvceciMLP/"
runs_path = "runs_synapsesbi/bg_CVAIF_EEIE_T4wvceciMLP/"
path_to_sim_config = "./tasks_configs/bg_CVAIF_EEIE_T4wvceciMLP.yaml"
with open(path_to_sim_config, "r") as f:
    sim_params = yaml.load(f, Loader=yaml.Loader)
    
lower_lim=torch.tensor([0., 0., -1., -1., -1., -1., #etaEE, etaIE, WpreEE, WpostEE, WpreIE, WpostIE
            -1., -1., -1., -1., -1., -1.,
            -1., -1., -1., -1., -1., -1.,
            -1., -1., -1., -1.])
upper_lim=torch.tensor([1., 1., 1., 1., 1., 1., #etaEE, etaIE, WpreEE, WpostEE, WpreIE, WpostIE
            1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1.])

metrics = ["rate"]

In [ ]:
## Choose which samples to use for the fit
dataset = load_and_merge(data_path, ("bg_CVAIF_EEIE_T4wvceciMLP_all.npy",))

cond_r = ("rate", 3, 15)

cond_cv = ("cv_isi", 0.7, 1000)
cond_sf = ("spatial_Fano", 0.5, 2.5)
cond_tf = ("temporal_Fano", 0.5, 2.5)
cond_ac = ("auto_cov", 0, 0.1)
cond_fft = ("fft", 0, 1)
cond_wb = ("w_blow", 0, 0.1)
cond_srt = ("std_rate_temporal", 0, 0.5)
cond_srs = ("std_rate_spatial", 0, 5)
cond_scv = ("std_cv", 0, 0.2)
cond_wc = ("w_creep", 0, 0.05)
cond_ri = ("rate_i", 1, 50)
cond_weef =("weef", 0 ,0.5)
cond_wief =("wief", 0 ,5)

condition = apply_n_conditions(dataset_aux, (cond_r,cond_ri,
            cond_wb,cond_wc,cond_weef,cond_wief,
            cond_ac,cond_cv,cond_fft,cond_srt,cond_srs,cond_sf,cond_tf))

dataset = dataset_aux[condition]
print(str(np.sum(condition)) + "/" + str(len(condition)), "samples kept for training")

#### 2/ Fit a posterior with the sbi package

In [ ]:
# prepare the training set for the posterior: [theta, xs]
thetas = torch.tensor(dataset['theta'][:,:-1], dtype=torch.float32) #remove nuisance parameter from training (input rate)
xs = torch.tensor([[dataset[i][j] for i in metrics] for j in range(len(dataset))], dtype=torch.float32)

# some requirements for using the sbi package, not useful in our specific case
bounds = {'low':lower_lim,
          'high':upper_lim}
prior = torch.distributions.Uniform(low=lower_lim, high=upper_lim)

In [ ]:
# train and save the posterior: this can take time (~1h) depending on your hardware. If too long you can include fewer samples in the training set (at the cost of a potentially worse posterior)
mk_post = MakePosterior(**sim_params["prior_params"])
mk_post.get_ensemble_posterior(
    thetas,
    xs,
    save_path=runs_path + "posterior_" + round_name + ".pkl")

Congratulations, you now have a posterior you can sample new rules from!     

Head to the Sample_from_posterior jupyter notebook to do that.

#### Bonus: initial round (starting fSBI from scratch)

###### If round 0: draw samples from prior, WITHOUT classifier

In [ ]:
data_path = "data_synapsesbi/bg_IF_EEEIIEII_6pPol/"

### POLY
prior_cls = "_Prior_bg_IF_EEEIIEII_6pPol"
prior_cls_args = dict(lower_lim=[0.01, 0.01, -2., -2., -2., -2.,
                         0.01, 0.01, -2., -2., -2., -2.,
                         0.01, 0.01, -2., -2., -2., -2.,
                         0.01, 0.01, -2., -2., -2., -2.,
                         5],
              upper_lim=[.1, .1, 2., 2., 2., 2.,
                         .1, .1, 2., 2., 2., 2.,
                         .1, .1, 2., 2., 2., 2.,
                         .1, .1, 2., 2., 2., 2.,
                         15]
                     )

## MLP
# prior_cls = "_Prior_bg_CVAIF_EEIE_T4wvceciMLP"
# prior_cls_args = dict(lower_lim=[0., 0., -1., -1., -1., -1., #etaEE, etaIE, WpreEE, WpostEE, WpreIE, WpostIE
#                 -1., -1., -1., -1., -1., -1.,
#                 -1., -1., -1., -1., -1., -1.,
#                 -1., -1., -1., -1., 5],
#     upper_lim= [1., 1., 1., 1., 1., 1., #etaEE, etaIE, WpreEE, WpostEE, WpreIE, WpostIE
#                 1., 1., 1., 1., 1., 1.,
#                 1., 1., 1., 1., 1., 1.,
#                 1., 1., 1., 1., 15])



prior = Prior(prior_class=prior_cls,
              prior_class_args=prior_cls_args,
              return_numpy=False,
              restrict_prior=False,
              )

num_samples = 10
thetas, seeds = _make_unique_samples((num_samples,),
                                     prior=prior,
                                     saved_seeds=[])

###### If round 0+: draw samples from prior + classifier

We train a classifier on the training set to accept or reject new samples (for sample efficiency).  
This is done only during the inner iterations of round 0 for sample efficiency.  

See this paper for more info: https://projecteuclid.org/ebooks/institute-of-mathematical-statistics-lecture-notes-monograph-series/A-Festschrift-for-Herman-Rubin/chapter/Generalized-Accept-Reject-sampling-schemes/10.1214/lnms/1196285403

In [ ]:
## load training data for classifier

## POLY
for_classifier = np.load(data_path + "box_prior_metrics.npy")

## MLP
# for_classifier = np.load(save_dir + "box_prior_metrics.npy")

theta_classifier, x_classifier = FT(for_classifier['theta']), FT(for_classifier['rate'])
sum(~torch.isfinite(x_classifier)) / len(x_classifier)